# # Plotting evaluation results.

In [ ]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import platform
import matplotlib as mpl
import random
from copy import copy
import re

# if platform.system() == 'Darwin':
#     matplotlib.use("TkAgg")

import matplotlib.animation
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import scipy as sp
import pprint

from pathlib import Path

from relnet.utils.general_utils import *
from relnet.agent.baseline.baseline_agent import *
from relnet.agent.mcts.mcts_agent import *

from relnet.evaluation.storage import EvaluationStorage
from relnet.evaluation.experiment_conditions import *
from relnet.evaluation.file_paths import FilePaths
from relnet.evaluation.experiment_conditions import *
from relnet.visualization import *


algorithm_class = 'planning'

exp_ids_prelim = ['prelim_kh_25', 'prelim_kh_50', 'prelim_kh_75']
exp_ids_sg_synth = ['sg_uct_synth_kh_25', 'sg_uct_synth_kh_50', 'sg_uct_synth_kh_75']
#exp_ids_sg_synth = []

exp_ids_btm = ['btm_kh_25', 'btm_kh_50', 'btm_kh_75']
exp_ids_mincost = ['mincost_kh_25', 'mincost_kh_50', 'mincost_kh_75']
exp_ids_reduction = ['reduction_kh_25', 'reduction_kh_50', 'reduction_kh_75']

exp_ids_rw = ['sg_uct_rw']

storage = EvaluationStorage()
fp = FilePaths('/experiment_data', 'aggregate', setup_directories=True)


considered_agents_baseline = [RandomAgent,
                       GreedyAgent,
                       CostSensitiveGreedyAgent,
                       LowestDegreeProductAgent,
                       FiedlerVectorAgent,
                       EffectiveResistanceAgent,
                       MinCostAgent,
                       LBHBAgent
                             ]

considered_agents_baseline_rw = [RandomAgent,
                       LowestDegreeProductAgent,
                       FiedlerVectorAgent,
                       EffectiveResistanceAgent,
                       MinCostAgent,
                       LBHBAgent
                             ]

exp_ids_t1 = exp_ids_prelim + exp_ids_sg_synth
exp_ids_ablation = exp_ids_btm + exp_ids_mincost + exp_ids_reduction

considered_agents_t1 = [StandardMCTSAgent, SGUCTAgent]
#considered_agents_t1 = [StandardMCTSAgent]
considered_agents_ablation = [BTMMCTSAgent, MinCostMCTSAgent, AR80RandMCTSAgent, AR60RandMCTSAgent, AR40RandMCTSAgent,
                             AR40DegreeMCTSAgent, AR40InvDegreeMCTSAgent, AR40LBHBMCTSAgent, AR40NumConnsMCTSAgent, 
                             AR40BestEdgeMCTSAgent, AR40BestEdgeCSMCTSAgent, AR40AvgEdgeMCTSAgent, AR40AvgEdgeCSMCTSAgent]

considered_agents_rw = [StandardMCTSAgent, SGUCTAgent]

considered_agents_hyps = [StandardMCTSAgent, MinCostMCTSAgent, SGUCTAgent]

all_agents_t1 = considered_agents_baseline + considered_agents_t1
all_agents_ablation = [StandardMCTSAgent] + considered_agents_ablation
all_agents_rw = considered_agents_baseline_rw + considered_agents_rw

display_order_t1 = [a.algorithm_name for a in all_agents_t1] # + ['uct_ratios']
display_order_ablation = [a.algorithm_name for a in all_agents_ablation]
display_order_rw = [a.algorithm_name for a in all_agents_rw]
display_order_hyps = [a.algorithm_name for a in considered_agents_rw]


In [ ]:
def fetch_results(exp_ids):
    all_result_dfs = []
    for exp_id in exp_ids: 
        results = storage.get_evaluation_data(algorithm_class, exp_id)
        experiment_details = storage.get_experiment_details(algorithm_class, exp_id)
        # hyps = storage.retrieve_optimal_hyperparams(algorithm_class, exp_id, {}, experiment_details['experiment_conditions']['train_individually'])
        # print(f"optimal hyps for <<{exp_id}>> were:")
        # pprint.pprint(hyps)

        results_df = pd.DataFrame(results)
        results_df.dropna(inplace=True)
        all_result_dfs.append(results_df)

    agg_df = pd.concat(all_result_dfs)
    return agg_df

def create_pivot(results_df, all_agents, add_mcts_ratios=False, which='synth'):
    nondet_agents = [a.algorithm_name for a in all_agents if not a.is_deterministic]
    pivot_idxes = {
        'synth': ['objective_function', 'network_size'],
        'rw_agg': ['objective_function', 'network_generator'],
        'rw_full': ['objective_function', 'network_generator', 'graph_id']
    }
    
    pivot_idx = pivot_idxes[which]
    pivot = pd.pivot_table(results_df, values='cummulative_reward', 
                       index=pivot_idx, 
                       columns=['algorithm'],
                       aggfunc=np.mean)

    nondet_df = results_df[results_df['algorithm'].isin(nondet_agents)]
    nondet_means_df = pd.pivot_table(nondet_df, values='cummulative_reward', 
                           columns=['algorithm', 'agent_seed'], 
                           index=pivot_idx,                       
                           aggfunc=np.mean)

    format_ci_dict = {}
    for agent_name in nondet_agents:
        cis = nondet_means_df[agent_name].apply(compute_ci, axis=1)
        pivot[agent_name + "_ci"] = cis
        format_ci_dict[agent_name + "_ci"] = (lambda x: "±{:.3f}".format(abs(x)))

    format_ci_dict = {}
    for agent in all_agents:
        if agent.is_deterministic:
            continue
        agent_name = agent.algorithm_name
        cis = nondet_means_df[agent_name].apply(compute_ci, axis=1)
        pivot[agent_name + "_ci"] = cis
        format_ci_dict[agent_name + "_ci"] = (lambda x: "±{:.3f}".format(abs(x)))
        
    if add_mcts_ratios:
        uct_40 = AR40MCTSAgent.algorithm_name
        uct_full = NoARMCTSAgent.algorithm_name
        pivot['uct_ratios'] = pivot[uct_40] / pivot[uct_full]

    pivot.style.format("{:.3f}").format(format_ci_dict)
    return pivot
    
def clean_and_write_latex(pivot_table, all_agents, cols_order, filename, transpose=True, which='results'):
    latex_df = pivot_table.copy()
    if which != 'hyps':
        nondet_agents = [a.algorithm_name for a in all_agents if not a.is_deterministic]

        for nondet_agent in nondet_agents:
            colname_ci = f"{nondet_agent}_ci"
            latex_df[nondet_agent] = latex_df.agg(lambda x: f"{x[nondet_agent]:.3f}±{x[colname_ci]:.3f}", axis=1)
            latex_df.drop(columns=[colname_ci], inplace=True)

        latex_df = latex_df[cols_order]
        row_maxes = latex_df.max(axis=1)

    repl_cols = copy(agent_display_names)
    repl_cols = {k:v for (k,v) in repl_cols.items() if not "UCT" in v}
    
    latex_df.rename(columns=repl_cols, inplace=True)
    latex_df.replace(objective_function_display_names, inplace=True)
    
    
    
    texfile =  str(fp.figures_dir / filename)
    fh = open(texfile, 'w')
    if transpose:
        latex_df = latex_df.T
        #t_colnames = list(latex_df.columns)
        #table_colformat = f"c|{''.join(['c'] * len(t_colnames)) }"
        #latex_df.to_latex(buf=fh, float_format="{:0.3f}".format, column_format=table_colformat)
        latex_df.to_latex(buf=fh, float_format="{:0.3f}".format)
        fh.close()
    else:
        print(f"which is {which}.")
        if which == 'hyps':
            print(f"assigning under hyps")
            #table_colformat = f"lll|{''.join(['l'] * len(cols_order)) }"
            latex_df.to_latex(buf=fh, float_format="{:0.3f}".format)
            fh.close()
        else:
            if which == 'rw_agg':
                table_colformat = f"cc|{''.join(['c'] * len(cols_order)) }"
            elif which in ['rw_full', 'hyps']:
                table_colformat = f"ccc|{''.join(['c'] * len(cols_order)) }"
            else:
                raise ValueError(f"which {which} not recognised.")
            latex_df.to_latex(buf=fh, float_format="{:0.3f}".format, column_format=table_colformat)
            fh.close()

    replace_dict = {
        r"reduction\\_policy": r"$\phi$",
        r"sim\\_policy\\_bias": r"$\\beta$",
        r"exp\\_name": r"Experiment",
        r"synth\\_25": r"KH-25",
        r"synth\\_50": r"KH-50",
        r"synth\\_75": r"KH-75",
        r"rw\\_internet\\_topology": r"Internet",
        r"rw\\_metro": r"Metro",        
        r"rw": "Real-world",
        
        r"C\\_p": r"$C_p$",
        
        r"uct\\_btm": r"SG-UCT\\textsubscript{BTM}",
        r"uct\\_mincost": r"SG-UCT\\textsubscript{MINCOST}",

        r"uct\\_rand\\_80": r"SG-UCT\\textsubscript{RAND-80}",
        r"uct\\_rand\\_60": r"SG-UCT\\textsubscript{RAND-60}",
        r"uct\\_rand\\_40": r"SG-UCT\\textsubscript{RAND-40}",        
        
        r"uct\\_deg\\_40": r"SG-UCT\\textsubscript{DEG-40}",
        r"uct\\_invdeg\\_40": r"SG-UCT\\textsubscript{INVDEG-40}",
        r"uct\\_lbhb\\_40": r"SG-UCT\\textsubscript{LBHB-40}",
        r"uct\\_numconns\\_40": r"SG-UCT\\textsubscript{NC-40}",
        r"uct\\_be\\_40": r"SG-UCT\\textsubscript{BE-40}",        
        r"uct\\_becs\\_40": r"SG-UCT\\textsubscript{BECS-40}",
        r"uct\\_ae\\_40": r"SG-UCT\\textsubscript{AE-40}",
        r"uct\\_aecs\\_40": r"SG-UCT\\textsubscript{AECS-40}",        

        r"sg\\_uct": r"SG-UCT",        
        r"uct": r"UCT",
        
        r"greedy\\_cs": r"Greedy\\textsubscript{CS}",
        
        r"avg\\_edge\\_cs": r"AECS-40",
        
        r"uct\\_ratios": r"UCT\\textsubscript{40} / UCT",
        
        r"internet\\_topology": "Internet",
        r"metro": "Metro",
        r"graph\\_id": "Graph",
        
        r"nan±nan": r"---",
        r"NaN": r"---",
        r"nan": r"---",
        r"objective\\_function": r"Objective",
        r"network\\_size" : r"$|V|$",
        r"network\\_generator" : r"$\\mathbf{G}$",
        r"kaiser\\_hilgetag": r"KH",
        r"internet\\_topology": r"Internet",
        r"metro": r"Metro",
        r"agent": r"Agent",
        r"algorithm" : r"",
        r"global\\_eff": r"$\\mathcal{F}_{E}$",
        r"lcs\\_targeted": r"$\\mathcal{F}_{R}$",
        r"±(\d+\.\d+)": r"\\tiny{$\\pm\g<1>$}",
        r"±---": r"\\tiny{$\\pm0.000$}"
    }

    with open(texfile, 'r') as f:
        raw_content = f.read()

    processed_content = raw_content
    for orig, targ in replace_dict.items():
        processed_content = re.sub(orig, targ, processed_content, flags = re.M)

    with open(texfile, 'w') as g:
        g.write(processed_content)

# Experiment 1: Preliminary Results + SG-UCT

In [ ]:
t1_results = fetch_results(exp_ids_t1)
t1_results.head(5)

In [ ]:
pivot = create_pivot(t1_results, all_agents_t1, add_mcts_ratios=False, which='synth')
clean_and_write_latex(pivot, all_agents_t1, display_order_t1, f't1_results.tex', which='synth')
pivot.T

# Experiment 2: Ablation Study

In [ ]:
ablation_results = fetch_results(exp_ids_ablation)
ablation_results = pd.concat([ablation_results, t1_results[t1_results['algorithm'] == StandardMCTSAgent.algorithm_name]])
ablation_results.head(5)

In [ ]:
pivot = create_pivot(ablation_results, all_agents_ablation, which='synth')
clean_and_write_latex(pivot, all_agents_ablation, display_order_ablation, 'ablation_results.tex', which='synth')
pivot

# Figure: mincost policy bias

In [ ]:
hyperparam_dfs = {}
agent_name = MinCostMCTSAgent.algorithm_name

for experiment_id in exp_ids_mincost:
    param_spaces, df = storage.get_hyperparameter_optimisation_data(algorithm_class, experiment_id, {}, train_individually=False)

    latest_experiment = storage.get_experiment_details(algorithm_class, experiment_id)
    objective_functions = latest_experiment["objective_functions"]
    num_nodes = latest_experiment['experiment_conditions']['base_n']

    expanded_data = []
    for objective_function in objective_functions:
        subset = df[(df['agent_name'] == agent_name) & (df['objective_function'] == objective_function)]
        subset.drop(columns=['agent_name'])

        for idx, row in subset.iterrows():
            row_copy = dict(row)
            hyperparams_id = row['hyperparams_id']
            hyperparams = param_spaces[objective_function][agent_name][hyperparams_id]
            row_copy.update(hyperparams)
            expanded_data.append(row_copy)

        hyp_df = pd.DataFrame(expanded_data).drop(columns=['hyperparams_id', 'graph_id', 'network_generator'])
        hyp_df['N'] = [str(num_nodes)] * len(hyp_df)
        hyp_df = hyp_df.rename(columns={"sim_policy_bias": r"$\beta$", "avg_reward": "Mean Reward"})
        
        if objective_function not in hyperparam_dfs:
            hyperparam_dfs[objective_function] = []
        
        hyperparam_dfs[objective_function].append(hyp_df)

fig_title = 'mincost_beta.pdf'        
figure_save_path = fp.figures_dir / fig_title
plot_beta_param(hyperparam_dfs, figure_save_path)


# Table: Real-World Graph Results

In [ ]:
rw_results = fetch_results(exp_ids_rw)
rw_results.head(5)

In [ ]:
pivot = create_pivot(rw_results, all_agents_rw, which='rw_full')
clean_and_write_latex(pivot, all_agents_rw, display_order_rw, 'rw_results_full.tex', transpose=False, which='rw_full')

In [ ]:
pivot = create_pivot(rw_results, all_agents_rw, which='rw_agg')
clean_and_write_latex(pivot, all_agents_rw, display_order_rw, 'rw_results_agg.tex', transpose=False, which='rw_agg')

# Table: Hyperparameters Used

In [ ]:
all_exps = exp_ids_t1 + exp_ids_mincost + exp_ids_rw

relevant_hyps = ['C_p', 'sim_policy_bias', 'reduction_policy']
exp_id_to_name = {('prelim_kh_25', 'kaiser_hilgetag'): 'synth_25',
                  ('prelim_kh_50', 'kaiser_hilgetag'): 'synth_50',
                  ('prelim_kh_75', 'kaiser_hilgetag'): 'synth_75',
                  
                  ('mincost_kh_25', 'kaiser_hilgetag'): 'synth_25',
                  ('mincost_kh_50', 'kaiser_hilgetag'): 'synth_50',
                  ('mincost_kh_75', 'kaiser_hilgetag'): 'synth_75',
                  
                  ('sg_uct_synth_kh_25', 'kaiser_hilgetag'): 'synth_25',
                  ('sg_uct_synth_kh_50', 'kaiser_hilgetag'): 'synth_50',
                  ('sg_uct_synth_kh_75', 'kaiser_hilgetag'): 'synth_75',
                  
                  ('sg_uct_rw', 'internet_topology'): 'rw_internet_topology',
                  ('sg_uct_rw', 'metro'): 'rw_metro'
                 }

bootstrap_id_map = {
      'mincost_kh_25': 'prelim_kh_25',
      'mincost_kh_50': 'prelim_kh_50',
      'mincost_kh_75': 'prelim_kh_75',
    
}

hyps_rows = []


for exp_id in all_exps: 
    print(exp_id)
    experiment_details = storage.get_experiment_details(algorithm_class, exp_id)
    gens = experiment_details['network_generators']
    objs = experiment_details['objective_functions']
    is_individual = experiment_details['experiment_conditions']['train_individually']
    hyps = storage.retrieve_optimal_hyperparams(algorithm_class, exp_id, {}, is_individual)
    print(hyps)

    for gen in gens:
        for obj in objs:
            for agent in considered_agents_hyps:
                if not is_individual:
                    if (gen, obj, agent.algorithm_name) in hyps:
                        hyps_vals = hyps[(gen, obj, agent.algorithm_name)][0]

                        entry = {}
                        entry['exp_name'] = exp_id_to_name[(exp_id, gen)]
                        entry['objective_function'] = obj
                        entry['agent'] = agent.algorithm_name
                        entry['graph_id'] = '---'

                        for hyp_name in relevant_hyps:
                            if hyp_name == 'C_p' and agent.algorithm_name == MinCostMCTSAgent.algorithm_name:
                                boostrapped_hyps = storage.retrieve_optimal_hyperparams(algorithm_class, bootstrap_id_map[exp_id], {}, experiment_details['experiment_conditions']['train_individually'])
                                entry[hyp_name] = str(boostrapped_hyps[(gen, obj, StandardMCTSAgent.algorithm_name)][0][hyp_name])
                                print(f"woo, entry is {hyp_name, entry[hyp_name]}")
                            else:
                                if hyp_name in hyps_vals:
                                    entry[hyp_name] = str(hyps_vals[hyp_name])
                                else:
                                    if agent.algorithm_name == SGUCTAgent.algorithm_name and hyp_name == 'sim_policy_bias':
                                        # specified as default parameter for SG-UCT
                                        entry[hyp_name] = str(25)
                                    else:
                                        entry[hyp_name] = '---'
                        hyps_rows.append(entry)
                else:
                    gen_class = retrieve_generator_class(gen)
                    gids = get_graph_ids_to_iterate(is_individual, gen_class, fp)
                    print(f"gids were {gids}")
                    for gid in gids:
                        if (gen, obj, agent.algorithm_name, gid) in hyps:
                            hyps_vals = hyps[(gen, obj, agent.algorithm_name, gid)][0]
                            print(hyps_vals)
                            entry = {}
                            entry['exp_name'] = exp_id_to_name[(exp_id, gen)]
                            entry['objective_function'] = obj
                            entry['agent'] = agent.algorithm_name
                            entry['graph_id'] = gid
                            for hyp_name in relevant_hyps:
                                if hyp_name in hyps_vals:
                                    entry[hyp_name] = str(hyps_vals[hyp_name])
                                else:
                                    if agent.algorithm_name == SGUCTAgent.algorithm_name and hyp_name == 'sim_policy_bias':
                                        # specified as default parameter for SG-UCT
                                        entry[hyp_name] = str(25)
                                    else:
                                        entry[hyp_name] = '---'

                            hyps_rows.append(entry)

                    
                        
hyps_df = pd.DataFrame(hyps_rows)
hyps_df.head(25)
    

In [ ]:
pivot = pd.pivot_table(hyps_df, values=relevant_hyps,
                   index= ['exp_name', 'graph_id', 'agent'],
                   columns=['objective_function'],
                   aggfunc='first')
pivot
clean_and_write_latex(pivot, considered_agents_hyps, display_order_hyps, 'hyps.tex', transpose=False, which='hyps')